
## Passwords Auditor with contraints


In [1]:

## !pip install zxcvbn
## !pip install password-strength
## !pip install passlib


In [2]:


import string
import torch
import torch.nn as nn
import torch.nn.functional as F
from zxcvbn import zxcvbn
from password_strength import PasswordStats
import math


In [3]:


VOCAB = list(string.ascii_letters + string.digits + "!@#$%^&*")
VOCAB 


['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '!',
 '@',
 '#',
 '$',
 '%',
 '^',
 '&',
 '*']

In [4]:

V     = len(VOCAB)
V


70

In [5]:

MASK_UPPER = torch.tensor([c in string.ascii_uppercase for c in VOCAB]).float()

print(  MASK_UPPER.shape  )
MASK_UPPER


torch.Size([70])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:

MASK_LOWER = torch.tensor([c in string.ascii_lowercase for c in VOCAB]).float()

print( MASK_LOWER.shape )
MASK_LOWER


torch.Size([70])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:

MASK_DIGIT = torch.tensor([c in string.digits          for c in VOCAB]).float()

print( MASK_DIGIT.shape )
MASK_DIGIT


torch.Size([70])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [8]:



MASK_SPEC  = torch.tensor([c in "!@#$%^&*"     for c in VOCAB]).float()

print(MASK_SPEC.shape)

MASK_SPEC



torch.Size([70])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:


'''


class TinyDecoder(nn.Module):
    
    def __init__(self, z_dim=128, T=12, V=V):
        super().__init__()
        self.T = T
        self.V = V
        self.linear = nn.Linear(z_dim, T * V)

    def forward(self, z):
        logits = self.linear(z)                        # [B, T*V]
        return logits.view(z.size(0), self.T, self.V)  # [B, T, V]



'''


'\n\n\nclass TinyDecoder(nn.Module):\n    \n    def __init__(self, z_dim=128, T=12, V=V):\n        super().__init__()\n        self.T = T\n        self.V = V\n        self.linear = nn.Linear(z_dim, T * V)\n\n    def forward(self, z):\n        logits = self.linear(z)                        # [B, T*V]\n        return logits.view(z.size(0), self.T, self.V)  # [B, T, V]\n\n\n\n'

In [10]:


def estimate_entropy(password):
    charset_size = 0
    
    if any(c.islower() for c in password): charset_size += 26
    if any(c.isupper() for c in password): charset_size += 26
    if any(c.isdigit() for c in password): charset_size += 10
    if any(c in "!@#$%^&*()-_=+[]{};:'\",.<>/?\\|" for c in password): charset_size += 32
    return len(password) * math.log2(charset_size or 1)




In [11]:

def entropy_stats(passwords):
    entropies = [ estimate_entropy(pw) for pw in passwords ]
    return {
        "avg_entropy_bits": sum(entropies) / len(entropies),
        "min": min(entropies),
        "max": max(entropies)
    }


In [12]:

def average_entropy(passwords):
    
    return sum(estimate_entropy(p) for p in passwords) / len(passwords)



In [13]:

def score_with_password_strength(passwords):
    stats = [PasswordStats(pw) for pw in passwords]
    return {
        "average_strength": sum(s.strength() for s in stats) / len(stats),
        "weak%": sum(s.strength() < 0.3 for s in stats) / len(stats) * 100,
        "strong%": sum(s.strength() > 0.7 for s in stats) / len(stats) * 100
    }



In [14]:

def class_present(p_pos):
    
    # p_pos: [B, T] probability per position of being in the class
    
    return 1.0 - torch.prod(1.0 - p_pos + 1e-6, dim=1)      # [B]



In [15]:

# --- Decode one-hot/probabilities to strings ---

def decode(y_probs):
    idx = y_probs.argmax(dim=-1).cpu()                       # [B, T]
    out = []
    for row in idx.tolist():
        out.append("".join(VOCAB[i] for i in row))
    return out


In [16]:

def entropy_bits_per_string(y_probs):
    
    # y_probs: [B, T, V], probs per token
    
    p   = y_probs.clamp_min(1e-9)
    H_t = -(p * p.log()).sum(dim=-1)                         # nats, [B, T]
    H   = H_t.mean(dim=1)                                      # [B]
    
    return H / torch.log(torch.tensor(2.0))                  # bits



In [17]:

def generate_passwords(WpU,WpL,WpD,WpS,Wentr,batch, steps=300, T=12, H_min_bits=6.0, seed=0, use_gumbel=True):
    
    torch.manual_seed(seed)
    
    z_init = torch.full((batch, T, V), -6.0)
    
    ## breaks with z_init
    ## but losses still back prop to "z", thanks to requires_grad=true
    
    z      = z_init.clone().detach().requires_grad_(True)  

    opt    = torch.optim.Adam([z], lr=0.05)

    for step in range(steps):
        
        tau = max(0.8 - 0.003 * step, 0.2)
        
        if use_gumbel and step < 500:
            
            gumbel_noise = -torch.empty_like(z).exponential_().log()
            
            logits       = (z + gumbel_noise) / tau
            
            y_probs      = F.softmax(logits, dim=-1)
            
        else:
            y_probs = F.softmax(z / tau, dim=-1)
            
        
        pU = (y_probs * MASK_UPPER).sum(dim=-1)
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        class_loss = (
            WpU*F.relu( 1.0 - class_present(pU) ).mean() +
            WpL*F.relu( 1.0 - class_present(pL) ).mean() +
            WpD*F.relu( 1.0 - class_present(pD) ).mean() +
            WpS*F.relu( 1.0 - class_present(pS) ).mean()
        )

        H_bits = entropy_bits_per_string(y_probs)
        
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        loss = class_loss + entropy_loss*Wentr
        
        ## loss = class_loss + entropy_loss

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode( y_probs.detach() )



In [18]:


def evaluate_passwords(password_list):
   
    scores = []
    crack_times = []

    for pw in password_list:
        result = zxcvbn(pw)
        scores.append(result['score'])
        crack_times.append(result['crack_times_seconds']['offline_fast_hashing_1e10_per_second'])

    avg_score = sum(scores) / len(scores)
    score_distribution = {
        score: scores.count(score) for score in range(5)
    }

    return {
        'average_score': avg_score,
        'score_distribution': score_distribution,
        'avg_crack_time_secs': sum(crack_times) / len(crack_times)
    }


In [19]:

def eval_func_metrics(constraintsGeneratedPasswords):
    i = 0
    for pass_generated in constraintsGeneratedPasswords[12:32]:
        print(pass_generated)
    
    result = evaluate_passwords(constraintsGeneratedPasswords)
    
    print("Average score:", result['average_score'])
    print("Score distribution:", result['score_distribution'])
    print("Average crack time (seconds):", result['avg_crack_time_secs'])
    
    pw = constraintsGeneratedPasswords

    print(score_with_password_strength(pw))
    print(average_entropy(pw))
    print(entropy_stats(pw))
    



## Experiments


In [20]:

num_passwords  = 32     ## 2048
entropy_weight = 0.0



## All constraints


In [21]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=1.0,
                                     WpD=1.0,
                                     WpS=1.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)


p%69BFh0%9Y%
97$v@Cv3$x@V
z3j23#^H7^5O
P^#uW7TY&7r7
c!H6NWGKj$60
y&$*3nvL!3HB
m#03c&&#4ZR$
D#T13jg$#C&^
4uE*$3@@E7$%
lb@v5W6J02^$
H*$YC5hQ9#@u
6AbA5902*$z^
H8Q^%k#43dR^
@1Cfl0XH1!%8
cA70!80Z&^2d
&#2b66uh$5@C
2%5z6*24bb1O
3&2T&i6&02Eq
y!$P$xv!8^x@
#s^LX6%8^m4V
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 100.0000000001
{'average_strength': 0.509377100519998, 'weak%': 0.0, 'strong%': 0.0}
78.65506622013164
{'avg_entropy_bits': 78.65506622013164, 'min': 78.65506622013166, 'max': 78.65506622013166}



## No Special Chars


In [22]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=1.0,
                                     WpD=1.0,
                                     WpS=0.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)


io7214h0VxY8
hu213177oUsH
qBl8kW5F7g38
fjHqs6T2d50V
9g6EW86SRnwd
7b07185L5SPc
W21b02jbMTD4
BhT17qr3185p
DaNcG3q91m3v
7iWc1W4JrZtk
N35n850dJm8u
xyKCtW01UpB9
D30w1FygE8qY
Xv6aAo8318F0
cvBTF4f7r719
U2Pwp811Zk9Z
9va1hN1n0bT8
3VwdY9jH2Ilj
6XZmL9a83hRd
6sd8e5f8CQA9
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 94.7350000000875
{'average_strength': 0.5151182840132217, 'weak%': 0.0, 'strong%': 0.0}
71.45035572464252
{'avg_entropy_bits': 71.45035572464252, 'min': 71.45035572464249, 'max': 71.45035572464249}



## No Digits


In [23]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=1.0,
                                     WpD=0.0,
                                     WpS=1.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)


p$Kx@^G&wx$!
o$$W@qE^*!*V
!C@OvRN%*dd!
&#!u$!C@%Urk
L!G#X%kS#tn&
h%yY*&SL^xNn
m##$!^&$AT&G
j@d$#RpB#NCV
&!VcsE&u##@y
lsSVZR&*e*!m
#Y^CwqYk&s^#
E^OVp@quL$@^
I*V*^tZjMY!$
&vL*l&^eI!OX
cGO!#!N#&lO*
Ofv#yC*R^*#%
R&^v@*jBb&RK
L$p!$lp$geK^
**O^$&zcALx*
vgmL%T%&X!$Z
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 100.0000000001
{'average_strength': 0.5015357420003721, 'weak%': 0.0, 'strong%': 0.0}
76.70780907334513
{'avg_entropy_bits': 76.70780907334513, 'min': 76.70780907334513, 'max': 76.70780907334513}



## No lower case letters


In [24]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=0.0,
                                     WpD=1.0,
                                     WpS=1.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)



&2K91*5%%WY4
1$2#77E7#L@K
%&427#6^571O
&%B3S0F8!91V
@!B67V6#Z6C%
W%$PU%S@*^21
4323!5*KO*@@
2&S1X3%1%8R^
@J&5S5!&Y@$0
$G&G!N4F^7K&
N37C05DP!19@
5E^AB903@5!8
J4C*^9%4S3#$
!58!^&8H5&78
BK1YI#&#&!3!
A42@83@M^*@C
5@R1E#64#&1K
1^2!VM6$62#%
6%75$T535EU*
*85L3*@1&N4V
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 91.796000031340625
{'average_strength': 0.4981816256947674, 'weak%': 0.0, 'strong%': 0.0}
73.04955409500411
{'avg_entropy_bits': 73.04955409500411, 'min': 73.04955409500407, 'max': 73.04955409500407}



## No upper case letters


In [25]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=0.0,
                                     WpL=1.0,
                                     WpD=1.0,
                                     WpS=1.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)



9%50&!52$tj#
1$#^#j5k&9%7
!150p#$^#7r0
m&#8f7%!$78q
k!t8#%x!g54%
y&$5%zt*%4p*
j9^3e94b*ll4
w!pk66r@^8&8
6u^c33q60*@6
mb@^45@&59#$
@3@3&1mk&!@$
4^#ux902yoo^
m43m@m#80ne$
!%4z&@&h9@68
fiv*5&34#!3w
&9248u*6$xut
823z3z00!$4#
3!7p*^$&82@j
l*j5#*3$85l7
^s@189@803&3
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 89.5533125938375
{'average_strength': 0.49566434051168784, 'weak%': 0.0, 'strong%': 0.0}
73.04955409500411
{'avg_entropy_bits': 73.04955409500411, 'min': 73.04955409500407, 'max': 73.04955409500407}



## No special characters and No digits


In [26]:

constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=1.0,
                                     WpD=0.0,
                                     WpS=0.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)


aTiTyuLfVtAs
hkgEJqYUoLYT
qQgEzRQXtabq
PCWxxXzWxNrC
ByyPNnNytgSD
lhmPdIdLsZNs
sAgdCmSqSZHN
BhcUFvkZmnob
qSTEAoQkjEvj
msAsRATuRatH
HLqkBOYzNLHq
PkGPzUpFMzim
PkxhjFIoEYxY
HdnmVoLYrEBg
npNTFgyvXRCU
bTWXbpCEkZDi
zjdzcQWLrbVK
FPrAxABKTjzC
xMIRLetNhJJd
ghphmtEbVNbP
Average score: 4.0
Score distribution: {0: 0, 1: 0, 2: 0, 3: 0, 4: 32}
Average crack time (seconds): 95.0518750000875
{'average_strength': 0.5140620948926758, 'weak%': 0.0, 'strong%': 0.0}
68.40527661769309
{'avg_entropy_bits': 68.40527661769309, 'min': 68.4052766176931, 'max': 68.4052766176931}



## No special characters, and No digits, and No lower case letters


In [27]:


constraintsGeneratedPasswords = generate_passwords(
                                     WpU=1.0,
                                     WpL=0.0,
                                     WpD=0.0,
                                     WpS=0.0,
                                     Wentr=entropy_weight,
                                     batch=num_passwords, 
                                     steps=2000, 
                                     T=12, 
                                     H_min_bits=3.0
)



eval_func_metrics(constraintsGeneratedPasswords)


NPKIGFZGTCWJ
RSZLHDDQOXWH
CZJLETTHOZDT
USLRRXUBJBEP
BXXQZWYKZAOQ
DRTXZNVFJSRJ
FVCYMGFZGFDP
DOTQOLHFNMMP
BLHTHXBCCOXO
OJHUCXCCIMSB
IQMSUEBMBZZJ
PDCIPEDPXCIF
DYDGPAKQLUCB
JLXGBIBABCPY
DQCUORNUEPBZ
NTFTFBAUOQSF
QVCTIQTCRCBU
UBJDQZQISMEN
KCDRHCHQCXYB
VTGYDPBOTEZC
Average score: 3.96875
Score distribution: {0: 0, 1: 0, 2: 0, 3: 1, 4: 31}
Average crack time (seconds): 84.53193753131875
{'average_strength': 0.48399943143790947, 'weak%': 0.0, 'strong%': 0.0}
56.405276617693076
{'avg_entropy_bits': 56.405276617693076, 'min': 56.405276617693104, 'max': 56.405276617693104}
